# Create Coastline Segments

This notebook modifies the CoDEC points to generate a uniform set of coastline segment centroids. It then filters the Natural Earth coastlines to only those line segments that surround a body of land containing physical capital and/or population OR those segments that contain a modified CoDEC point.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import geopandas as gpd
import pandas as pd
from shapely import box
from shapely.geometry import Polygon
from sliiders import settings as sset
from sliiders import spatial
from sliiders.io import read_shapefile, save_geoparquet

spatial.filter_spatial_warnings()

In [3]:
# Import GTSM station point that have been snapped to the NatEarth coastlines layer
gtsm0 = pd.read_parquet(sset.PATH_GEOG_GTSM_SNAPPED)

# Import Europe stations to be thinned (came in 5x higher res than rest of world)
# Stations to be thinned were manually ID'ed in GIS
gtsm_e0 = pd.read_parquet(sset.PATH_GEOG_GTSM_STATIONS_TOTHIN)
gtsm_e0["serial_id"] = gtsm_e0.station_id.str[-5:]
gtsm_e0 = gtsm_e0.sort_values("serial_id")

# Filter 80% of Europe stations
gtsm1 = gtsm0.drop(gtsm_e0.drop(gtsm_e0.index[::5]).station_id)

# Add back in Gibraltar and Monaco
gib_id = "id_coast_glob_eur_03452"
mon_id = "id_coast_glob_eur_03236"

# add in better centered points for Akrotiri and Dhekelia (XAD)
ak_id = "id_coast_glob_eur_00108"
ak_id_drop = "id_coast_glob_eur_00107"

de_id = "id_coast_glob_eur_00097"
de_id_drop = "id_coast_glob_eur_00096"

gtsm1 = pd.concat(
    [gtsm1.drop([ak_id_drop, de_id_drop]), gtsm0.loc[[gib_id, mon_id, ak_id, de_id]]]
)
assert gtsm1.index.is_unique

In [4]:
# Import manual points to add for small country coastlines missing from GTSM
add_pts = pd.read_parquet(sset.PATH_SEG_PTS_MANUAL, columns=["lon", "lat"])

add_pts.index = pd.Index(
    ["id_coast_glob_990" + str(i + 1).zfill(2) for i in range(len(add_pts))],
    name="station_id",
)

gtsm1 = pd.concat([gtsm1, add_pts])

In [5]:
# Manual deletions of certain points that don't fall nearer to coastlines than other
# points
del_ids = [
    "eur_01019",
    "eur_01812",
    "eur_00979",
    "13536",
    "14447",
    "15646",
    "18265",
    "18656",
    "18720",
    "18724",
]
del_ids = ["id_coast_glob_" + del_ids[i] for i in range(len(del_ids))]
gtsm1 = gtsm1.drop(del_ids)

In [6]:
# Remove buoy and obs points and duplicates
gtsm1 = gtsm1[
    ~(gtsm1.index.str.contains("buoy") | gtsm1.index.str.contains("obs"))
].drop_duplicates()

In [7]:
# reformat as geodataframe
gtsm1 = gpd.GeoDataFrame(
    gtsm1.assign(geometry=gpd.points_from_xy(gtsm1.lon, gtsm1.lat)), crs="epsg:4326"
)
assert gtsm1.index.is_unique

In [9]:
# Export final set of majority GTSM points for CIAM segs
save_geoparquet(gtsm1, sset.PATH_SEG_CENTROIDS)

# Create Coastlines from Natural Earth Layer Using GTSM points and Exposure Grid

In [8]:
# Import Natural Earth Coastlines
coastline_polys = read_shapefile(sset.PATH_NATEARTH_LANDPOLYS)
coastline_polys = (
    coastline_polys[coastline_polys.featurecla.ne("Null island")]
    .geometry.explode(index_parts=False)
    .reset_index(drop=True)
)

# drop antarctica
coastline_polys = coastline_polys[coastline_polys.bounds.miny > -60]

# drop caspian
caspian = coastline_polys.interiors.explode().dropna()
assert len(caspian) == 1
ix = caspian.index[0]
coastline_polys[ix] = Polygon(coastline_polys[ix].exterior)

# convert to coastlines
coastlines = coastline_polys.boundary

In [79]:
save_geoparquet(coastlines.to_frame("geometry"), sset.PATH_NATEARTH_COASTLINES_INT)

## Intersect land polys with exposure grid

In [10]:
# Define intersection function
# Width, in degrees, of cell to which exposure cells are rounded
# If there is exposure anywhere within the cell, this notebook
# assumes the entire cell is covered by exposure
ROUNDED_BOX_SIZE = 1 / 10


def get_no_exp_coastlines(exp, cell_size):

    exp["y_ix"] = spatial.grid_val_to_ix(
        spatial.grid_ix_to_val(exp.y_ix.values, cell_size=cell_size),
        cell_size=ROUNDED_BOX_SIZE,
    )
    exp["x_ix"] = spatial.grid_val_to_ix(
        spatial.grid_ix_to_val(exp.x_ix.values, cell_size=cell_size, lon_mask=True),
        cell_size=ROUNDED_BOX_SIZE,
        lon_mask=True,
    )

    cells = exp[["x_ix", "y_ix"]].drop_duplicates().reset_index(drop=True)

    lons = spatial.grid_ix_to_val(cells["x_ix"].values, cell_size=ROUNDED_BOX_SIZE)
    lats = spatial.grid_ix_to_val(cells["y_ix"].values, cell_size=ROUNDED_BOX_SIZE)

    boxes = box(
        lons - (ROUNDED_BOX_SIZE / 2),
        lats - (ROUNDED_BOX_SIZE / 2),
        lons + (ROUNDED_BOX_SIZE / 2),
        lats + (ROUNDED_BOX_SIZE / 2),
    )

    box_gdf = gpd.GeoDataFrame(geometry=boxes, crs=4326)

    matches = gpd.sjoin(coastline_polys.to_frame("geometry"), box_gdf, how="left")

    return matches.index[matches["index_right"].isnull()].unique()

In [11]:
# Intersect and store flag for exposure/no-exposure
no_value_line_ids = get_no_exp_coastlines(
    pd.read_parquet(sset.PATH_EXPOSURE_ASSET_VALUE_BLENDED), sset.ASSET_VALUE_GRID_WIDTH
)
no_pop_line_ids = get_no_exp_coastlines(
    pd.read_parquet(sset.PATH_EXPOSURE_POP_INT).reset_index(), sset.POP_GRID_WIDTH
)

no_exp_line_ids = no_pop_line_ids.union(no_value_line_ids)

coastline_polys = coastline_polys.to_frame().assign(
    exposure=~coastline_polys.index.isin(no_exp_line_ids)
)

/srv/conda/envs/notebook/lib/python3.11/site-packages/pygeos/io.py:89: UserWarning: The shapely GEOS version (3.12.1-CAPI-1.18.1) is incompatible with the PyGEOS GEOS version (3.10.4-CAPI-1.16.2). Conversions between both will be slow
  warnings.warn(


## Intersect land polys with snapped GTSM points

In [12]:
coastline_polys = coastline_polys.rename_axis("line_id").reset_index()

In [13]:
# Create tiny buffer of GTSM points for intersection with land polys
gtsm_buff = (
    gpd.GeoDataFrame(gtsm1.buffer(0.001))
    .rename(columns={0: "geometry"})
    .set_geometry("geometry")
)
gtsm_land_int = gpd.overlay(
    coastline_polys, gtsm_buff, how="intersection", keep_geom_type=True
)

# Set flag in polys layer for containing GTSM point or not
coastline_polys["gtsm_pt"] = coastline_polys.line_id.isin(
    gtsm_land_int.line_id.unique()
)

## Create Final Coastlines layer filtered by having non-zero exposure or a GTSM point 

In [14]:
coastlines_filt = coastlines.loc[
    coastline_polys.loc[
        coastline_polys.exposure | coastline_polys.gtsm_pt, "line_id"
    ].values
]

In [21]:
save_geoparquet(
    coastlines_filt.to_frame("geometry"), sset.PATH_GEOG_COASTLINES, index=False
)